In [ ]:
using JLD2
using FileIO
using PyPlot
using Statistics
using StatsBase 
using Distributions
using LinearAlgebra
using DSP
using HDF5

In [ ]:
using IntervalSets
using Random, ArraysOfArrays
using ValueShapes
using Measurements
using BAT

In [ ]:
include("../model-41/likelihood.jl")
include("../model-41/utill_model.jl")
include("../utill.jl")

In [ ]:
conv_mat = load("../../data/experiment/dataset_2/m2/conv-matrix-upd-2.jld2")

conv_matrices = (
    cam_1 = conv_mat["cam_1"],
    cam_2 = conv_mat["cam_2"],
    cam_3 = conv_mat["cam_3"],
    cam_4 = conv_mat["cam_4"],
);


In [ ]:
param_truth = (
    tr_size = [0.145, 0.145], 
    tr_size_2 = [0.10, 0.10], 
    ang_spr = [6, 6], 
    ang_spr_2 = [3, 3], 
    mixt_pow = 0.7, 
    waist = [3.15], 
    waist_2 = [2.9], 
    algmx = [36.0, 37.0, 19.0, 33.0], 
    algmy = [33.0, 34.0, 21.0, 35.0], 
    cam4_ped = 26.0, 
    cam4_light_fluct = 1.50, 
    cam4_light_amp = 8.90, 
    resx = [1, 1, 1], 
    resy = [1, 1, 1], 
    cam4_resx = 3.0, 
    cam4_resy = 3.0, 
    psx = [27.1, 21.6, 114.0], 
    psy = [30.5, 23.4, 125.0], 
    cam4_psx = 121.8, 
    cam4_psy = 120.0, 
    light_amp = [8.5, 11.1, 2.5], 
    s_cam = [0.0, 1.478, 15.026, 23.115]
);

In [ ]:
event = generate_event(param_truth, 27.0, conv_matrices)

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(7,7))
fig.subplots_adjust(hspace=0.2, wspace=0.2)

ax[1,1].pcolormesh(event.cam_1, cmap="RdYlBu_r")
ax[1,2].pcolormesh(event.cam_2, cmap="RdYlBu_r")
ax[2,1].pcolormesh(event.cam_3, cmap="RdYlBu_r")
ax[2,2].pcolormesh(event.cam_4, cmap="RdYlBu_r")

In [ ]:
# @save "../../data/sampling_results/Benchmark-6/event-41.jld" event

In [ ]:
β1= 0.015
β2 = 0.0077
β3 = 0.0058 

# prior = NamedTupleDist(
#         tr_size = [truncated(Normal(0.145, 0.04), 0.06, 0.19), truncated(Normal(0.145, 0.04), 0.06, 0.19)],
#         tr_size_2 = [truncated(Normal(0.145, 0.04), 0.06, 0.19), truncated(Normal(0.145, 0.04), 0.06, 0.19)],
#         ang_spr = [truncated(Normal(6, 2.0), 4.0, 7.0), truncated(Normal(6.0, 2.0), 4.0, 7.0)],
#         ang_spr_2 = [truncated(Normal(3.0, 2.0), 1.0, 4.0), truncated(Normal(3.0, 2.0), 1.0, 4.0)],
#         mixt_pow =  0.50 .. 1.0 ,
#         waist = [truncated(Normal(3.15, 0.03), 2.65, 3.5)],
#         waist_2 = [truncated(Normal(2.9, 0.03), 2.65, 3.5)], # 11
#         algmx = [23.0 .. 48, 23.0 .. 48.0, 10.0 .. 30.0, 23.0 .. 48.0],
#         algmy = [23.0 .. 48, 23.0 .. 48.0, 10.0 .. 30.0, 23.0 .. 48.0],
#         cam4_ped = 4.0 .. 40.0,
#         cam4_light_fluct = 1.0 .. 3.0,
#         cam4_light_amp = 1.6 .. 9.9, 
#         resx = [1, 1, 1], # 23, 24, 25, 
#         resy = [1, 1, 1], # 26,27, 28, 
#         cam4_resx = truncated(Normal(3, 1.5), 0, Inf),
#         cam4_resy = truncated(Normal(3, 1.5), 0, Inf), 
#         psx = [27.1, 21.6, 114.0], # 31, 32, 33
#         psy = [30.5, 23.4, 125.0], # 34, 35, 36
#         cam4_psx = 121.8, # 37
#         cam4_psy = 120.0, # 38
#         light_amp  = [1.0 .. 13.0 , 1.0 .. 17.0, 1.0 .. 5.0], # 1.0 .. 5.0
#         s_cam = [0.0, 1.478, 15.026, 23.1150],
#     );

prior = NamedTupleDist(
        tr_size = [0.05 .. 0.2, 0.05 .. 0.2],
        tr_size_2 = [0.05 .. 0.2, 0.05 .. 0.2],
        ang_spr = [5 .. 7, 5 .. 7],
        ang_spr_2 = [1 .. 4, 1 .. 4],
        mixt_pow =  0.50 .. 1.0 ,
        waist = [3.0 .. 3.3],
        waist_2 = [2.8 .. 3.0], # 11
        algmx = [36.0, 37.0, 19.0, 33.0], 
        algmy = [33.0, 34.0, 21.0, 35.0], 
        cam4_ped = 26.0, 
        cam4_light_fluct = 1.50, 
        cam4_light_amp = 8.90, 
        resx = [1, 1, 1], 
        resy = [1, 1, 1], 
        cam4_resx = 3.0, 
        cam4_resy = 3.0, 
        psx = [27.1, 21.6, 114.0], 
        psy = [30.5, 23.4, 125.0], 
        cam4_psx = 121.8, 
        cam4_psy = 120.0, 
        light_amp = [8.5, 11.1, 2.5], 
        s_cam = [0.0, 1.478, 15.026, 23.115]
    );

In [ ]:
"""
    Log-Likelihood of the beamline (4 cameras included)
"""
function log_lik_ndiff(;  
    e = event,
    cv_mat = conv_matrices,
    func = conv_tabl_discrete,
    cv_1 = cv_mat.cam_1,
    cv_2 = cv_mat.cam_2,
    cv_3 = cv_mat.cam_3,
    n_threads = Threads.nthreads(),
    )
    
    return params -> begin 
        ll = 0.0
        ll += likelihood_cam13(params, e.cam_1, e.population, cv_1, func, 1, n_threads=n_threads)
        ll += likelihood_cam13(params, e.cam_2, e.population, cv_2, func, 2, n_threads=n_threads)
        ll += likelihood_cam13(params, e.cam_3, e.population, cv_3, func, 3, n_threads=n_threads)
        ll += likelihood_cam4(params, e.cam_4, e.population, 4, n_threads=n_threads)
        return LogDVal(ll)
    end
        
end

log_likelihood = log_lik_ndiff()

In [ ]:
posterior = PosteriorDensity(log_likelihood, prior)

tuning = AdaptiveMHTuning(
    λ = 0.5,
    α = ClosedInterval(0.15,0.25),
    β = 1.5,
    c = ClosedInterval(1e-4,1e2),
    r = 0.5,
)

convergence = BrooksGelmanConvergence(
    threshold = 1.1,
    corrected = false
)

init = MCMCChainPoolInit(
    init_tries_per_chain = ClosedInterval(50,150),
    max_nsamples_init = 500,
    max_nsteps_init = 500,
    max_time_init = Inf
)

burnin = MCMCMultiCycleBurnin(
    max_nsamples_per_cycle = 10000,
    max_nsteps_per_cycle = 10000,
    max_time_per_cycle = Inf,
    max_ncycles = 130
)

nsamples = 4*10^5
nchains = 4

sampler = MetropolisHastings(tuning=tuning,)

algorithm = MCMCSampling(sampler=sampler, 
    nchains=nchains, 
    init=init, 
    burnin=burnin, 
    convergence=convergence
);

In [ ]:
@time samples_tot = bat_sample(
    posterior, nchains*nsamples, algorithm,
    max_neval = nchains*nsamples,
    max_time = Inf,
);

In [ ]:
samples = samples_tot.result;

In [ ]:
minimum(bat_eff_sample_size(unshaped.(samples)).result)

In [ ]:
size(flatview(unshaped.(samples.v)))

In [ ]:
plot(flatview(unshaped.(samples.v))[3,:])

In [ ]:
plt.imshow(log.(abs.(cov(flatview(unshaped.(samples.v))'))), cmap="Blues")

In [ ]:
# BAT.bat_write("../../data/sampling_results/Benchmark-6/samples-41.hdf5", unshaped.(samples))

In [ ]:
flat_truth = vcat([i for i in param_truth]...)
flat_mode = vcat([i for i in mode(samples)[1]]...) ./flat_truth
flat_std = vcat([i for i in std(samples)[1]]...)./flat_truth;

In [ ]:
plt.errorbar(1:length(flat_mode), flat_mode, yerr=flat_std )

In [ ]:
bat_marginalmode(samples).result

In [ ]:
mean(samples)[1]

In [ ]:
std(samples)[1]

In [ ]:
dims = [1,2,3,4,5,6,7,8,9,10,11]
# dims = [1,2,3,4,5,]
names = [1,2,3,4,5,6,7,8,9,10, 11]
# levels = [0.01, 0.55, 0.9, 0.95,  1,]
levels = [0.001, 0.90, 1,]
figsize = (10,10)
n_bins = 20

corner_plots(samples, dims, names, levels_quantiles = levels, figsize=figsize, N_bins=n_bins)

In [ ]:
dims = [10,11,]
names = [10,11,]
levels = [0.1, 0.8, 0.9, 0.95,  1,]
figsize = (6,6)
n_bins = 20

corner_plots(samples, dims, names, levels_quantiles = levels, figsize=figsize, N_bins=n_bins)

In [ ]:
data = [sort(randn(500)) for i in 1:5];
quant = [(0.1, 0.9) for i in 1:5]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(7,7))

parts = ax.violinplot(data, widths = 0.9, vert=true, showmeans=true, showextrema=true, quantiles=quant)

for pc in parts["bodies"]
    pc.set_facecolor("gray")
    pc.set_edgecolor("black")
    pc.set_alpha(0.5)
    pc.set_linewidth(2)
end

In [ ]:
? plt.violinplot

In [ ]:
? plt.collections.PolyCollection